# Check GPU

In [1]:
import torch
print(torch.cuda.device_count()) 
print(torch.cuda.is_available())
print(torch.__version__)
print(torch.cuda.current_device())

1
True
1.13.1
0


# Fine-tune Whisper

## Prepare Data and Model

### Load Model

In [2]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

### Load Datasets

In [3]:
AUDIO_COLUMN_NAME = "audio"
TEXT_COLUMN_NAME = "sentence"

In [4]:
from datasets import Audio

def normalize_dataset(ds, audio_column_name=None, text_column_name=None):

    if audio_column_name is not None and audio_column_name != AUDIO_COLUMN_NAME:
        ds = ds.rename_column(audio_column_name, AUDIO_COLUMN_NAME)

    if text_column_name is not None and text_column_name != TEXT_COLUMN_NAME:
        ds = ds.rename_column(text_column_name, TEXT_COLUMN_NAME)

    # resample to the same sampling rate
    ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

    # normalise columns to ["audio", "sentence"]
    ds = ds.remove_columns(set(ds.features.keys()) - set([AUDIO_COLUMN_NAME, TEXT_COLUMN_NAME]))
    
    return ds

### Use huggingface API

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from datasets import load_dataset, DatasetDict

datasets = DatasetDict()

In [8]:
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "fr", split="test", use_auth_token=True)
dataset = normalize_dataset(dataset)

Computing checksums of downloaded files. They can be used for integrity verification. You can disable this by passing ignore_verifications=True to load_dataset


Computing checksums:  11%|#1        | 2/18 [00:09<01:15,  4.74s/it]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Generating other split: 0 examples [00:00, ? examples/s]





Reading metadata...: 14359it [00:00, 224294.58it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]
















Reading metadata...: 57607it [00:00, 228340.74it/s]


Dataset common_voice_11_0 downloaded and prepared to /home/user/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/fr/11.0.0/2c65b95d99ca879b1b1074ea197b65e0497848fd697fdb0582e0f6b75b6f4da0. Subsequent calls will reuse this data.


In [9]:
dataset

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 16089
})

### Split dataset into train and test

In [10]:
num_train_examples = 200
num_test_examples = 50

datasets["train"] = dataset.select(range(num_train_examples))
datasets["eval"] =  dataset.select(range(num_test_examples))

In [11]:
datasets

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 200
    })
    eval: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 50
    })
})

# Data Augmentation

In [13]:
!pip install audiomentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.1 MB/s eta 0:00:00
  Using cached librosa-0.9.2-py3-none-any.whl (214 kB)
  Using cached resampy-0.4.2-py3-none-any.whl (3.1 MB)
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0
    Uninstalling librosa-0.10.0:
      Successfully uninstalled librosa-0.10.0


In [14]:
from audiomentations import (
    AddBackgroundNoise,
    AddGaussianNoise,
    Compose,
    Gain,
    OneOf,
    PitchShift,
    PolarityInversion,
    TimeStretch,
)

### define augmentation

In [15]:
augmentation = Compose(
    [
        TimeStretch(min_rate=0.9, max_rate=1.1, p=0.2, leave_length_unchanged=False),
    ]
)

In [16]:
def augment_dataset(batch):
    # load and (possibly) resample audio data to 16kHz
    sample = batch[AUDIO_COLUMN_NAME]

    # apply augmentation
    augmented_waveform = augmentation(sample["array"], sample_rate=sample["sampling_rate"])
    batch[AUDIO_COLUMN_NAME]["array"] = augmented_waveform
    return batch

### augment training data

In [20]:
preprocessing_num_workers = 1

augmented_raw_training_dataset = datasets["train"].map(
    augment_dataset, num_proc=preprocessing_num_workers, desc="augment train dataset"
)

augment train dataset:   0%|          | 0/200 [00:00<?, ?ex/s]

### Combine raw data and augment data

In [21]:
from datasets import concatenate_datasets

datasets["train"] = concatenate_datasets([datasets["train"], augmented_raw_training_dataset])
datasets["train"] = datasets["train"].shuffle(seed=10)

In [22]:
datasets

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 400
    })
    eval: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 50
    })
})

## Preprocessing

In [23]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

In [24]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="french", task="transcribe")

In [25]:
do_normalize_text = True


def prepare_dataset(batch):

    # load
    audio = batch[AUDIO_COLUMN_NAME]

    # compute log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    # process targets
    input_str = normalizer(batch[TEXT_COLUMN_NAME]).strip() if do_normalize_text else batch[TEXT_COLUMN_NAME]
    
    # encode target text to label ids
    batch["labels"] = processor.tokenizer(input_str).input_ids

    return batch

In [26]:
vectorized_datasets = datasets.map(
    prepare_dataset, remove_columns=next(iter(datasets.values())).column_names, desc="preprocess dataset"
)

preprocess dataset:   0%|          | 0/400 [00:00<?, ?ex/s]

preprocess dataset:   0%|          | 0/50 [00:00<?, ?ex/s]

## Training and Evaluation

### Data Collator

In [27]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]

        # convert to tensors
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        
        # pad label ids to the max length in the batch
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [28]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

In [29]:
import evaluate

metric = evaluate.load("wer")

In [30]:
# evaluate with the 'normalised' WER
do_normalize_eval = True


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if do_normalize_eval:
        pred_str = [normalizer(pred) for pred in pred_str]

        # perhaps already normalised
        label_str = [normalizer(label) for label in label_str]
        
        # filtering step to only evaluate the samples that correspond to non-zero references
        pred_str = [pred_str[i] for i in range(len(pred_str)) if len(label_str[i]) > 0]
        label_str = [label_str[i] for i in range(len(label_str)) if len(label_str[i]) > 0]

    wer = metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

### Training Configuration

In [72]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./outputs/whisper_medium_ft",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    warmup_steps=800,
    max_steps=8000,
    learning_rate=6.25e-6,
    weight_decay=0.01,
    gradient_checkpointing=True,
    fp16=True,
    predict_with_generate=True,
    generation_max_length=225,
    logging_steps=25,
    report_to=["tensorboard"],
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

PyTorch: setting up devices


### Initialize Trainer

In [73]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=vectorized_datasets["train"],
    eval_dataset=vectorized_datasets["eval"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


### GPU check again

In [74]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print(f"Using GPU is CUDA:{os.environ['CUDA_VISIBLE_DEVICES']}")

for i in range(torch.cuda.device_count()):
    info = torch.cuda.get_device_properties(i)
    print(f"CUDA:{i} {info.name}, {info.total_memory / 1024 ** 2}MB")

device = torch.device("cuda:0")

Using GPU is CUDA:1
CUDA:0 NVIDIA GeForce RTX 4090, 24195.125MB


### Training

In [75]:
train_result = trainer.train()

The following columns in the training set don't have a corresponding argument in `WhisperForConditionalGeneration.forward` and have been ignored: input_length. If input_length are not expected by `WhisperForConditionalGeneration.forward`,  you can safely ignore this message.
/home/user/miniconda3/envs/asr_new/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 400
  Num Epochs = 320
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8000
  Number of trainable parameters = 241734912


  0%|          | 0/8000 [00:00<?, ?it/s]

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

{'loss': 53.1181, 'learning_rate': 1.875e-07, 'epoch': 1.0}


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

...

  0%|          | 0/7 [00:00<?, ?it/s]

Saving model checkpoint to ./outputs/whisper_medium_ft/checkpoint-8000
Configuration saved in ./outputs/whisper_medium_ft/checkpoint-8000/config.json
Configuration saved in ./outputs/whisper_medium_ft/checkpoint-8000/generation_config.json


{'eval_loss': 0.008823619224131107, 'eval_wer': 14.549425287356321, 'eval_runtime': 20.6724, 'eval_samples_per_second': 2.419, 'eval_steps_per_second': 0.339, 'epoch': 320.0}


Model weights saved in ./outputs/whisper_medium_ft/checkpoint-8000/pytorch_model.bin
Feature extractor saved in ./outputs/whisper_medium_ft/checkpoint-8000/preprocessor_config.json
tokenizer config file saved in ./outputs/whisper_medium_ft/checkpoint-8000/tokenizer_config.json
Special tokens file saved in ./outputs/whisper_medium_ft/checkpoint-8000/special_tokens_map.json
added tokens file saved in ./outputs/whisper_medium_ft/checkpoint-8000/added_tokens.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./outputs/whisper_medium_ft/checkpoint-8000 (score: 14.549425287356321).


{'train_runtime': 16560.319, 'train_samples_per_second': 7.729, 'train_steps_per_second': 0.483, 'train_loss': 2.560914797578007, 'epoch': 320.0}


In [76]:
model.save_pretrained(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)

Configuration saved in ./outputs/whisper_medium_ft/config.json
Configuration saved in ./outputs/whisper_medium_ft/generation_config.json
Model weights saved in ./outputs/whisper_medium_ft/pytorch_model.bin
Feature extractor saved in ./outputs/whisper_medium_ft/preprocessor_config.json
tokenizer config file saved in ./outputs/whisper_medium_ft/tokenizer_config.json
Special tokens file saved in ./outputs/whisper_medium_ft/special_tokens_map.json
added tokens file saved in ./outputs/whisper_medium_ft/added_tokens.json


## Evaluation

In [77]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small",  language="french", task="transcribe")

loading file vocab.json from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6be735e3d386ffb60ad/vocab.json
loading file tokenizer.json from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6be735e3d386ffb60ad/tokenizer.json
loading file merges.txt from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6be735e3d386ffb60ad/merges.txt
loading file normalizer.json from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6be735e3d386ffb60ad/normalizer.json
loading file added_tokens.json from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6be735e3d386ffb60ad/added_tokens.json
loading file special_tokens_map.json from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6b

In [78]:
eval_metrics = trainer.evaluate(metric_key_prefix="eval")

eval_metrics

The following columns in the evaluation set don't have a corresponding argument in `WhisperForConditionalGeneration.forward` and have been ignored: input_length. If input_length are not expected by `WhisperForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 8


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.008823619224131107,
 'eval_wer': 14.549425287356321,
 'eval_runtime': 20.5723,
 'eval_samples_per_second': 2.43,
 'eval_steps_per_second': 0.34,
 'epoch': 320.0}

In [79]:
trainer.state.log_history

[{'loss': 53.1181, 'learning_rate': 1.875e-07, 'epoch': 1.0, 'step': 25},
 {'loss': 52.388, 'learning_rate': 3.828125e-07, 'epoch': 2.0, 'step': 50},
 {'loss': 50.7311, 'learning_rate': 5.78125e-07, 'epoch': 3.0, 'step': 75},
 {'loss': 48.0142,
  'learning_rate': 7.734375000000001e-07,
  'epoch': 4.0,
  'step': 100},
 {'loss': 45.4373, 'learning_rate': 9.375e-07, 'epoch': 5.0, 'step': 125},
 {'loss': 44.5122, 'learning_rate': 1.1328125e-06, 'epoch': 6.0, 'step': 150},
 {'loss': 43.2381, 'learning_rate': 1.328125e-06, 'epoch': 7.0, 'step': 175},
 {'loss': 41.1915, 'learning_rate': 1.5234375e-06, 'epoch': 8.0, 'step': 200},
 {'loss': 37.2746,
  'learning_rate': 1.7187500000000003e-06,
  'epoch': 9.0,
  'step': 225},
 {'loss': 28.0446,
  'learning_rate': 1.8984375000000002e-06,
  'epoch': 10.0,
  'step': 250},
 {'loss': 13.4677, 'learning_rate': 2.09375e-06, 'epoch': 11.0, 'step': 275},
 {'loss': 9.1882, 'learning_rate': 2.2890625e-06, 'epoch': 12.0, 'step': 300},
 {'loss': 7.6227,
  'lea

In [81]:
import pandas as pd
result = pd.DataFrame(trainer.state.log_history)

In [82]:
result.to_csv('Result_finetune_TimeStretch.csv')